In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

# Function to train the model for financial health scoring
def train_financial_health_model(csv_file):
    """
    Train a machine learning model for financial health scoring.

    Parameters:
    - csv_file (str): Path to the CSV file containing training data.

    Returns:
    - model: Trained RandomForestRegressor model.
    """
    # Load the dataset
    data = pd.read_csv(csv_file)

    # Ensure 'financial_health_score' exists or generate it
    if 'financial_health_score' not in data.columns:
        data["financial_health_score"] = np.random.randint(30, 100, size=len(data))  # Create dummy scores for training

    # Specify feature columns (make sure they are relevant to financial health)
    feature_columns = ['Income', 'MonthlyDebtPayments', 'DebtToIncomeRatio', 'LoanAmount', 'LoanDuration', 
                       'NumberOfDependents', 'SavingsAccountBalance', 'TotalAssets', 'NetWorth']
    
    X = data[feature_columns]
    y = data["financial_health_score"]

    # Feature Scaling using StandardScaler
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

    # Initialize the model
    model = RandomForestRegressor(random_state=42)

    # Hyperparameter tuning with GridSearchCV
    param_grid = {
        'n_estimators': [50, 100, 200],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }
    
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1, scoring='neg_mean_squared_error')
    grid_search.fit(X_train, y_train)

    print("Best parameters found: ", grid_search.best_params_)
    model = grid_search.best_estimator_

    # Cross-validation for better model evaluation
    cv_scores = cross_val_score(model, X_scaled, y, cv=5, scoring='neg_mean_squared_error')
    rmse_scores = np.sqrt(-cv_scores)
    print(f"Cross-validated RMSE: {rmse_scores.mean()}")

    # Train the model on the full training data with the best parameters
    model.fit(X_train, y_train)

    # Evaluate on test set
    y_pred = model.predict(X_test)
    rmse_test = np.sqrt(mean_squared_error(y_test, y_pred))
    print(f"Test RMSE: {rmse_test}")

    return model, scaler

# Function to predict financial health score based on user input
def predict_financial_health(model, scaler):
    """
    Predict the financial health score based on user input.

    Parameters:
    - model: The trained RandomForestRegressor model.
    - scaler: The fitted StandardScaler for feature scaling.
    """
    # User input for prediction
    income = float(input("Enter monthly income: "))
    monthly_debt = float(input("Enter monthly debt payments: "))
    debt_to_income_ratio = float(input("Enter debt-to-income ratio: "))
    loan_amount = float(input("Enter loan amount: "))
    loan_duration = float(input("Enter loan duration (in years): "))
    num_dependents = int(input("Enter number of dependents: "))
    savings_balance = float(input("Enter savings account balance: "))
    total_assets = float(input("Enter total assets: "))
    net_worth = float(input("Enter net worth: "))

    # Prepare the input data for prediction (in the same format as the training data)
    user_input = np.array([[income, monthly_debt, debt_to_income_ratio, loan_amount, loan_duration,
                            num_dependents, savings_balance, total_assets, net_worth]])

    # Scale the input data using the same scaler used during training
    user_input_scaled = scaler.transform(user_input)

    # Predict the financial health score
    prediction = model.predict(user_input_scaled)

    print(f"Your predicted Financial Health Score is: {prediction[0]}")

    # Provide advice based on the score
    if prediction < 40:
        print("Advice: Poor financial health. Consider cutting down on expenses and paying off high-interest debt.")
    elif prediction < 60:
        print("Advice: Fair financial health. Focus on saving and reducing unnecessary expenses.")
    elif prediction < 80:
        print("Advice: Good financial health. Keep saving and plan for future investments.")
    else:
        print("Advice: Excellent financial health! Consider diversifying investments and setting long-term financial goals.")

# Main function to run the program
def main():
    # Provide the CSV file path here
    csv_file = "/content/combined_dataset.csv"

    # Train the model and get the trained model and scaler
    model, scaler = train_financial_health_model(csv_file)

    # Ask the user whether they want to enter values or upload a PDF
    option = input("Would you like to input your financial details manually or upload a bank statement PDF? (Type 'manual' or 'pdf'): ").strip().lower()

    if option == 'manual':
        # If manual input, call the prediction function
        predict_financial_health(model, scaler)
    elif option == 'pdf':
        # If PDF upload, you can handle that separately (e.g., call the PDF extraction function)
        print("PDF upload functionality is not yet implemented.")
    else:
        print("Invalid option. Please enter 'manual' or 'pdf'.")

if __name__ == "__main__":
    main()
